In [ ]:
#КРЕДИТНЫЙ СКОРИНГ
#0. Описание проекта
#Построение модели, предсказывающей вероятность дефолта по кредиту на основе данных по клиенту


In [ ]:
#1. Импорт библиотек
# импортируем необходимые библиотеки
from pandas import Series
import pandas as pd
import numpy as np

import pandas_profiling

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import date
from datetime import datetime, timedelta

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, recall_score, f1_score, mean_absolute_error, mean_squared_error, precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import plot_confusion_matrix, plot_roc_curve, plot_precision_recall_curve

pd.set_option('display.max_rows', 50)  # показывать больше строк
pd.set_option('display.max_columns', 50)  # показывать больше колонок

import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
/kaggle/input/sf-dst-scoring/sample_submission.csv
/kaggle/input/sf-dst-scoring/train.csv
/kaggle/input/sf-dst-scoring/test.csv

In [ ]:
# Используемые функции

def show_roc_auc(y_test, y_probs):
    """Функция построения графика ROC AUC"""
    fpr, tpr, threshold = roc_curve(y_test, y_probs)
    roc_auc = roc_auc_score(y_test, y_probs)
    plt.figure()
    plt.plot([0, 1], label='Baseline', linestyle='--')
    plt.plot(fpr, tpr, label = 'Regression')
    plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc = 'lower right')
    plt.show()

    
def show_confusion_matrix(lastmodel,X_test,y_test):
    """Функция построения матрицы ошибок"""
    class_names = ['NonDefault', 'Default']
    titles_options = [("Confusion matrix", None)]
    for title, normalize in titles_options:
        disp = plot_confusion_matrix(lastmodel, X_test, y_test, 
                                     display_labels=class_names, 
                                     cmap=plt.cm.Blues, 
                                     normalize=normalize)
        disp.ax_.set_title(title)

        print(title)
        print(disp.confusion_matrix)

    plt.show()
    

def num_column_analysis(i):
    """Функция визуализации данных числовых признаков"""
    display(pd.DataFrame(train[i].value_counts(normalize=True, sort=True)))
    
    sns.boxplot(x = 'default', y = i, data = train) # строим boxplot
    plt.show()
    
    train[i].hist(bins = 100) # строим гистограмму
    
    
def combining_types_car(row):
    """Функция объединения признаков car и car_type"""
    result = row['car'] + row['car_type'] 
    return result
# в итоге получаем: если 0, то машины нет, если 1 - то есть отечественная, если 2 - то есть иномарка


def combining_types_adr(row):
    """Функция объединения признаков home_address и work_address"""
    result = 10*row['home_address'] + row['work_address'] 
    return result
# В итоге полуаем двузначное число, где первая цифра будет показывать признак home_address, вторая - work_address 

In [ ]:
#2. Загрузка и предварительный осмотр данных
PATH_to_file = '/kaggle/input/sf-dst-scoring/'
train = pd.read_csv(PATH_to_file + 'train.csv')
test = pd.read_csv(PATH_to_file + 'test.csv')
sample_submission = pd.read_csv(PATH_to_file + 'sample_submission.csv')
# Для контроля зафиксируем размер тренировочного и тестового датасетов.
print('Размер тренировочного датасета: ', train.shape,
      'Размер тестового датасета: ', test.shape, 
      'Размер объединенного датасета: ', train.shape[0]+test.shape[0], sep='\n')

In [ ]:
# Объединяем тренировочные и тестовые данные в один датасет для того чтоб все монипуляции с признаками проводить
# на обоих датасетах
train['train'] = 1 # помечаем тренировочные
test['train'] = 0 # помечаем тестовые
df = pd.concat([train, test], ignore_index=True)
df.sample(5)

In [ ]:
#Расшифровка признаков
#client_id - идентификатор клиента
#education - уровень образования
#sex - пол заемщика
#age - возраст заемщика
#car - флаг наличия автомобиля
#car_type - флаг автомобиля иномарки
#decline_app_cnt - количество отказанных прошлых заявок
#good_work - флаг наличия “хорошей” работы
#bki_request_cnt - количество запросов в БКИ
#home_address - категоризатор домашнего адреса
#work_address - категоризатор рабочего адреса
#income - доход заемщика
#foreign_passport - наличие загранпаспорта
#sna - связь заемщика с клиентами банка
#first_time - давность наличия информации о заемщике
#score_bki - скоринговый балл по данным из БКИ
#region_rating - рейтинг региона
#app_date - дата подачи заявки
#default - флаг дефолта по кредиту

In [ ]:
#Общая информация о полях
df.info()

In [ ]:
df.describe()

In [ ]:
# Проводим быстрый EDA c помощью pandas_profiling:
pandas_profiling.ProfileReport(df)

In [ ]:
#Из отчета видно что пропуски присутствуют только в колонке education. 
#Выборка по целевой переменной не сбалансирована. 
#Количество дефолтных клиентов в 6 раз больше чем не дефолтных. 
#По heatmap видно, что больше всего коррелируют между собой признаки: first_time и sna, work_address и home_address.

In [ ]:
#3. Предварительная обработка данных для построения наивной модели

# Начнем с того что избавимся от пропусков в колонке education.
# Посмотрим распределение значений.
df['education'].value_counts(normalize=True)


In [ ]:
# Заменим пропуски случайными значениями в том же соотношении, ACD пропускаем т.к. оно намного меньше 1%
df.fillna('empty',inplace=True)
df['education']=df['education'].apply(lambda x: np.random.choice(['SCH','GRD','UGR','PGR'],p=[0.53,0.32,0.13,0.02]) 
                                     if x=='empty' else x)

In [ ]:
# Создаем списки числовых, бинарных и категориальных переменных:
# Признак app_day пока выкидываем

# числовые
num_cols = [
    'age',
    'decline_app_cnt',
    'score_bki',
    'bki_request_cnt',
    'income',
    ]

# бинарные
bin_cols = ["sex", "car", "car_type", "good_work", "foreign_passport"]

# категориальные
cat_cols = [
    'education',
    'region_rating',
    'home_address',
    'work_address',
    'sna',
    'first_time',
    ]         
# Для бинарных признаков используем LabelEncoder

label_encoder = LabelEncoder()

for i in bin_cols:
    df[i] = label_encoder.fit_transform(df[i])
    
df[bin_cols].sample(5) # проверим что получилось

In [ ]:
# Разделим датасет обратно на обучающую и тестовую часть
train = df[df['train']==1].drop(['train'],axis=1)
test = df[df['train']==0].drop(['train','default'],axis=1)
# Проверим соответствие размеров датасетов исходным
print('Размер тренировочного датасета: ', train.shape,
      'Размер тестового датасета: ', test.shape, sep='\n')

In [ ]:
# Проведем OneHot-кодирование категориальных признаков:
X_cat = OneHotEncoder(sparse=False).fit_transform(train[cat_cols].values)
# Объединяем числовые, бинарные и категориальные переменные в одно признаковое пространство
X = np.hstack([train[num_cols], train[bin_cols].values,X_cat])
Y = train['default'].to_numpy().astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=32)

In [ ]:
#4. Построим наивную моель

# Обучаем модель:

model_0 = LogisticRegression(solver='liblinear')
model_0.fit(X_train, y_train)
y_pred = model_0.predict(X_test)

# Строим ROC AUC 

probs = model_0.predict_proba(X_test)
probs = probs[:,1]
show_roc_auc(y_test,probs)

# Строим матрицу ошибок

show_confusion_matrix(model_0,X_test,y_test)

# Остальные метрики

print('accuracy_score: {}'.format(np.round(accuracy_score(y_test, y_pred), 4)))
print('f1_score: {}'.format(np.round(f1_score(y_test, y_pred), 4)))
print('recall_score: {}'.format(np.round(recall_score(y_test, y_pred), 4)))

In [ ]:
#Результат оказался посредственным. Значение ROC AUC 0.55. 
#По матрице ошибок видно что модель вообще не предсказала дефолтных клиетов. Зато теперь есть на что опираться.

In [ ]:
#5. Дальнейшая обработка данных

# В столбце app_date переводим дату в подходящий вид:
df['app_date'] = pd.to_datetime(df['app_date'])

# Посмотрим период наблюдений:
display(df['app_date'].min())
display(df['app_date'].max())

In [ ]:
# Формируем новые признаки на основе даты:

df['day'] = df.app_date.dt.day
df['month'] = df.app_date.dt.month
df['weekday'] = df.app_date.dt.weekday
# Посчитаем количество дней до даты конца наблюдений
df['days'] = (df.app_date.max() - df.app_date).dt.days
# Избавляемся от уже не нужного столбца app_date
df.drop(['app_date'],axis=1, inplace=True)

In [ ]:
# Еще раз создаем списки числовых, бинарных и категориальных переменных с учетом изменений:

# числовые
num_cols = [
    'age',
    'decline_app_cnt',
    'score_bki',
    'bki_request_cnt',
    'income',
    'day',
    'month',
    'weekday',
    'days'
    ]

# бинарные
bin_cols = ["sex", "car", "car_type", "good_work", "foreign_passport"]

# категориальные
cat_cols = [
    'education',
    'region_rating',
    'home_address',
    'work_address',
    'sna',
    'first_time',
    ]         

In [ ]:
# посмотрим на корреляцию числовых признаков между собой
plt.figure(figsize=(9, 6))
sns.heatmap(df[num_cols].corr().abs(), vmin=0, vmax=1,
            annot=True, fmt=".2f", cmap="YlGnBu")

In [ ]:
#Как видим из вновь добавленных признаков сильно коррелируют days и month. От какого-то надо избавляться.

In [ ]:
# Посмотрим также на значимость числовых переменных:

temp = df[df['train'] == 1]
imp_num = pd.Series(f_classif(temp[num_cols], temp['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

In [ ]:
#Days находится выше, поэтому убираем month.
#Day и weekday не оказывают значимого влияния на целевую переменную. Их тоже удаляем.

df.drop(['month','day','weekday'],axis=1,inplace=True)
# Также удалим их из нашего списка числовых признаков
num_cols.remove('month')
num_cols.remove('day')
num_cols.remove('weekday')

In [ ]:
#Еще раз пройдемся признакам и посмотрим на них внимательнее. Начнем с простого, биномаильных признаков.

# Мы их уже перекодировали, так что посмотрим на их корреляцию
plt.figure(figsize=(9, 6))
sns.heatmap(df[bin_cols].corr().abs(), vmin=0, vmax=1,
            annot=True, fmt=".2f", cmap="YlGnBu")

In [ ]:
#Явно бросается в глаза сильная корреляция признаков car и car_type. Попробуем эти два признака объединить в один.

df['car'] = df.apply(lambda row: combining_types_car(row), axis=1)
# Удаляем уже не нужный столбец car_type
df.drop(['car_type'],axis=1,inplace=True)
bin_cols.remove('car_type')

In [ ]:
#Переходим к категориальным признакам. Посмотрим на их корреляцию.

plt.figure(figsize=(9, 6))
sns.heatmap(df[cat_cols].corr().abs(), vmin=0, vmax=1,
            annot=True, fmt=".2f", cmap="YlGnBu")

In [ ]:
#Здесь видна корреляция между пизнаками work_address и home_address. 
#Попробуем их также объединить в один признак. Между sna и first_time тоже не маленькая корреляция, 
#но думаю все же их оставить без изменения.

df['address'] = df.apply(lambda row: combining_types_adr(row), axis=1)

cat_cols.append('address')

cat_cols.remove('home_address')
df.drop(['home_address'], axis=1, inplace=True)

cat_cols.remove('work_address')
df.drop(['work_address'], axis=1, inplace=True)
# Разделим категориальные признаки по столбцам
df = pd.get_dummies(
    df, columns=['education', 'region_rating', 'sna', 'first_time', 'address','sex', 'car', 'good_work', 'foreign_passport'], dummy_na=False)

In [ ]:
#Остались только числовые признаки.

# Разделим наш общий датасет обратно на тренировочный и тестовый, чтоб проводить изменения только в тренировочном.
train = df[df['train']==1].drop(['train'],axis=1)
test = df[df['train']==0].drop(['train','default'],axis=1)

In [ ]:
# Проверим соответствие размеров датасетов исходным
print('Размер тренировочного датасета: ', train.shape,
      'Размер тестового датасета: ', test.shape, sep='\n')

In [ ]:
# Столбец age
num_column_analysis('age')

In [ ]:
#Выбросов нет, но распределение значений смещено в право. Прологарифмируем этот признак.

train['age'] = np.log(train['age'] + 1)
test['age'] = np.log(test['age'] + 1)

In [ ]:
# Столбец decline_app_cnt
num_column_analysis('decline_app_cnt')

In [ ]:
#Здесь уже есть выбросы, но они можно сказать единичны по сравнению со всем датасетом. 
#Больше значения 4 суммарное количество наблюдений не наберет и 1%. Поэтому все значения больше 4 заменим на 4.

train['decline_app_cnt']=train['decline_app_cnt'].apply(lambda x: x if x<=4 else 4)

In [ ]:
# столбец score_bki
num_column_analysis('score_bki')

In [ ]:
#Распределение нормальное. Выбросы хоть и есть, но думаю их оставить без изменений т.к. 
#их значения не сильно отличаются от остальных.

In [ ]:
# Столбец bki_request_cnt
num_column_analysis('bki_request_cnt')

In [ ]:
#Тут тоже наблюдаем выбросы. Т.к. их суммарная доля после значения 9 очень мала (меньше 1%), 
#то все что больше 9 заменим на 9. Распределение здесь смещено в право, прологарифмируем этот признак.

train['bki_request_cnt']=train['bki_request_cnt'].apply(lambda x: x if x<=9 else 9)
train['bki_request_cnt'] = np.log(train['bki_request_cnt'] + 1)
test['bki_request_cnt'] = np.log(test['bki_request_cnt'] + 1)

In [ ]:
# Столбец income
num_column_analysis('income')

In [ ]:
#Здесь тоже наблюдаем много выбросов. Больше 200к получают малое количество клиентов. 
#Приравняем их доход к 200к. Также прологорифмируем этот признак чтоб привести его к более нормальному распределению.

train['income']=train['income'].apply(lambda x: x if x<=200000 else 200000)
train['income'] = np.log(train['income'] + 1)
test['income'] = np.log(test['income'] + 1)

In [ ]:
#6. Построение модели

# Стандартизируем числовые признаки в обучающей и тестовой выборке
scaler=StandardScaler().fit(train[num_cols])
train_std=scaler.transform(train[num_cols])
train[num_cols]=train_std
test_std=scaler.transform(test[num_cols])
test[num_cols]=test_std
# Обновим общий датасет
train['train'] = 1 # помечаем тренировочные
test['train'] = 0 # помечаем тестовые
df = pd.concat([train, test], ignore_index=True)

# Делим выборку на обучающую и тестовую
X = train[list(set(train.columns) - set(['default','client_id']))].values
Y = train['default'].to_numpy().astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=32)

In [ ]:
# Обучаем модель:

model_1 = LogisticRegression(solver='liblinear')
model_1.fit(X_train, y_train)
y_pred = model_1.predict(X_test)

# Строим ROC AUC 

probs = model_1.predict_proba(X_test)
probs = probs[:,1]
show_roc_auc(y_test,probs)

# Строим матрицу ошибок

show_confusion_matrix(model_1,X_test,y_test)

# Остальные метрики

print('accuracy_score: {}'.format(np.round(accuracy_score(y_test, y_pred), 4)))
print('f1_score: {}'.format(np.round(f1_score(y_test, y_pred), 4)))
print('recall_score: {}'.format(np.round(recall_score(y_test, y_pred), 4)))

In [ ]:
#Уже лучше чем наивная модель. ROC AUC = 0.741. Модель хоть как-то начала определять дефолтных клиентов.

In [ ]:
#7. Модель 1 + oversampling

In [ ]:
# Произведем простейший oversampling посредством дублирования строк с дефолтом:

zeros = train[train['default'] == 0]
ones = train[train['default'] == 1]
default_new = int(len(zeros) / len(ones))
for i in range(default_new):
    train1 = train.append(ones).reset_index(drop=True)

In [ ]:
# Делим выборку на обучающую и тестовую
X = train1[list(set(train.columns) - set(['default','client_id']))].values
Y = train1['default'].to_numpy().astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=32)

In [ ]:
# Обучаем модель:

model_2 = LogisticRegression(solver='liblinear')
model_2.fit(X_train, y_train)
y_pred = model_2.predict(X_test)

# Строим ROC AUC 

probs = model_2.predict_proba(X_test)
probs = probs[:,1]
show_roc_auc(y_test,probs)

# Строим матрицу ошибок

show_confusion_matrix(model_2,X_test,y_test)

# Остальные метрики

print('accuracy_score: {}'.format(np.round(accuracy_score(y_test, y_pred), 4)))
print('f1_score: {}'.format(np.round(f1_score(y_test, y_pred), 4)))
print('recall_score: {}'.format(np.round(recall_score(y_test, y_pred), 4)))

In [ ]:
#Результат стал хуже. Площадь под кривой ROC AUC уменьшилась. 
#Количество ложно предсказанных не дефолтных клиентов тоже выросло.

In [ ]:
#8. Модель 1 + полиноминальные признаки

In [ ]:
# добавим новые признаки, через комбинацию
data=df.copy()
poly = PolynomialFeatures(2, include_bias=False)
poly_data = poly.fit_transform(data[num_cols])[:, len(num_cols):]
poly_cols = poly.get_feature_names()[len(num_cols):]
poly_df = pd.DataFrame(poly_data, columns=poly_cols)
data = data.join(poly_df,  how='left')

In [ ]:
# Разделяем датасет
train_p = data[data['train']==1].drop(['train'],axis=1)
test_p = data[data['train']==0].drop(['train','default'],axis=1)
# Делим выборку на обучающую и тестовую
X = train_p[list(set(train_p.columns) - set(['default','client_id']))].values
Y = train_p['default'].to_numpy().astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=32)

In [ ]:
# Обучаем модель:

model_3 = LogisticRegression(solver='liblinear')
model_3.fit(X_train, y_train)
y_pred = model_3.predict(X_test)

# Строим ROC AUC 

probs = model_3.predict_proba(X_test)
probs = probs[:,1]
show_roc_auc(y_test,probs)

# Строим матрицу ошибок

show_confusion_matrix(model_3,X_test,y_test)

# Остальные метрики

print('accuracy_score: {}'.format(np.round(accuracy_score(y_test, y_pred), 4)))
print('f1_score: {}'.format(np.round(f1_score(y_test, y_pred), 4)))
print('recall_score: {}'.format(np.round(recall_score(y_test, y_pred), 4)))

In [ ]:
#Площадь под кривой немного выросла. Матрица ошибок показывает примерно тот же результат.

In [ ]:
#9. Модель 1 + полиноминальные признаки + гиперпараметры

In [ ]:
# Делим выборку на обучающую и тестовую
X = train_p[list(set(train_p.columns) - set(['default','client_id']))].values
Y = train_p['default'].to_numpy().astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=32)

In [ ]:
# Зададим ограничения для параметра регуляризации
C = np.logspace(0, 4, 10)

penalty = ['l1','l2']
hyperparameters = dict(C=C, penalty=penalty)

model = LogisticRegression(solver = 'liblinear')
model.fit(X_train, y_train)

clf = GridSearchCV(model, hyperparameters, cv=5, verbose=0)

best_model = clf.fit(X_train, y_train)

print('Лучший penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Лучшее C:', best_model.best_estimator_.get_params()['C'])

In [ ]:
# Обучаем модель:

model_finish = LogisticRegression(penalty='l1', C=1.0, solver='liblinear')
model_finish.fit(X_train, y_train)
y_pred = model_finish.predict(X_test)

# Строим ROC AUC 

probs = model_finish.predict_proba(X_test)
probs = probs[:,1]
show_roc_auc(y_test,probs)

# Строим матрицу ошибок

show_confusion_matrix(model_finish,X_test,y_test)

# Остальные метрики

print('accuracy_score: {}'.format(np.round(accuracy_score(y_test, y_pred), 4)))
print('f1_score: {}'.format(np.round(f1_score(y_test, y_pred), 4)))
print('recall_score: {}'.format(np.round(recall_score(y_test, y_pred), 4)))

In [ ]:
#И даже с гиперпараметрами результат как-то не поменялся( 
#Модель так и продолжает предсказывать большое количество ложно не дефолтных клиентов.


In [ ]:
#10. Подготовка данных для соревнования

In [ ]:

X_test_fin = test_p[list(set(test_p.columns) - set(['default','client_id']))].values

y_probs = model_finish.predict_proba(X_test_fin)[:,1]

test_p['default'] = y_probs
submission = test_p[['client_id','default']]
display(submission.sample(10))
display(submission.shape)

submission.to_csv('submission.csv', index=False)